In [ ]:
%load_ext autoreload
%autoreload 2

Generate data

In [ ]:
import random

from common import constants
from schematic_generator import generator

configs = [
    # Simple shapes
    {
        "generator_type": ["shape"],
        "shape_type": ["sphere"],
        "radius": [lambda: random.randint(1, (constants.region_size[0] // 2) - 1)] * 5,
        "structure_block_types": [[block] for block in constants.simple_block_types] + [lambda: random.sample(constants.simple_block_types, 3)] * len(constants.simple_block_types),
        "background_block_types": [["minecraft:air"]],
        "position_offset": [lambda: (random.randint(-100, 100), random.randint(-100, 100), random.randint(-100, 100))],
        "random_seed": [lambda: random.randint(0, 2**32 - 1)],
        "region_size": [constants.region_size]
    },
    {
        "generator_type": ["shape"],
        "shape_type": ["cube"],
        "side_length": [lambda: random.randint(1, constants.region_size[0] - 1)] * 5,
        "structure_block_types": [[block] for block in constants.simple_block_types] + [lambda: random.sample(constants.simple_block_types, 3)] * len(constants.simple_block_types),
        "background_block_types": [["minecraft:air"]],
        "position_offset": [lambda: (random.randint(-100, 100), random.randint(-100, 100), random.randint(-100, 100))],
        "random_seed": [lambda: random.randint(0, 2**32 - 1)],
        "region_size": [constants.region_size]
    },
    # Filled
    {
        "generator_type": ["shape"],
        "shape_type": ["sphere"],
        "radius": [lambda: random.randint(3, (constants.region_size[0] // 2) - 1)] * 3,
        "structure_block_types": [[block] for block in constants.simple_block_types] + [lambda: random.sample(constants.simple_block_types, 3)] * (len(constants.simple_block_types) // 3),
        "structure_fill_block_types": [["minecraft:air"], lambda: random.sample(constants.simple_block_types, 1), lambda: random.sample(constants.simple_block_types, 3)],
        "thickness": [lambda: random.randint(1, 3)],
        "background_block_types": [["minecraft:air"]],
        "position_offset": [lambda: (random.randint(-100, 100), random.randint(-100, 100), random.randint(-100, 100))],
        "random_seed": [lambda: random.randint(0, 2**32 - 1)],
        "region_size": [constants.region_size]
    },
    {
        "generator_type": ["shape"],
        "shape_type": ["cube"],
        "side_length": [lambda: random.randint(7, constants.region_size[0] - 1)] * 3,
        "structure_block_types": [[block] for block in constants.simple_block_types] + [lambda: random.sample(constants.simple_block_types, 3)] * (len(constants.simple_block_types) // 3),
        "structure_fill_block_types": [["minecraft:air"], lambda: random.sample(constants.simple_block_types, 1), lambda: random.sample(constants.simple_block_types, 3)],
        "thickness": [lambda: random.randint(1, 3)],
        "background_block_types": [["minecraft:air"]],
        "position_offset": [lambda: (random.randint(-100, 100), random.randint(-100, 100), random.randint(-100, 100))],
        "random_seed": [lambda: random.randint(0, 2**32 - 1)],
        "region_size": [constants.region_size]
    }
]

generator.generate_samples_from_configurations(configs, dry_run=False)

Prepare data

In [ ]:
from model.data_preparer import prepare_data

prepare_data()

In [ ]:
import os

import h5py

from common.file_paths import TRAINING_DATA_DIR

with h5py.File(os.path.join(TRAINING_DATA_DIR, 'data.h5'), 'r') as hf:
    # Iterate over all groups in the root of the file
    for group_name in hf:
        group = hf[group_name]
        
        # Check if both "input_embedding" and "target_tensor" datasets exist in the current group
        if "input_embedding" in group and "target_tensor" in group:
            input_dataset = group["input_embedding"]
            target_dataset = group["target_tensor"]
            
            # Optionally, print their shapes to verify
            print(f"Group '{group_name}':")
            print(f"  Input Embedding Shape: {input_dataset.shape}")
            print(f"  Target Tensor Shape: {target_dataset.shape}")
        else:
            # If either dataset is missing, print a warning
            print(f"Warning: Group '{group_name}' is missing one or both of the required datasets.")
    
    # Optionally, print the total number of groups in the file
    print(f"Total Groups: {len(hf)}")

Train

In [1]:
import os
import re
import random

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, random_split
from torch.utils.tensorboard import SummaryWriter
from tqdm import tqdm

from common.file_paths import TRAINING_DATA_DIR
from model.dataset import MinecraftDataset
from model.model import MinecraftStructureGenerator

random.seed(0)
torch.manual_seed(0)
torch.cuda.manual_seed_all(0)

experiment_name = 'test14'

INPUT_EMBEDDING_SIZE = 1536
NUM_CLASSES = 345
OUTPUT_SIZE = [64, 64, 64]
model = MinecraftStructureGenerator(INPUT_EMBEDDING_SIZE, NUM_CLASSES, OUTPUT_SIZE)
print(model)
print(f"Number of parameters: {sum(p.numel() for p in model.parameters() if p.requires_grad)}")

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer, 'min', factor=0.1, patience=10, verbose=True)

# Define the directory where checkpoints are saved
checkpoint_dir = f'checkpoints/{experiment_name}'
os.makedirs(checkpoint_dir, exist_ok=True)

# List all checkpoint files
checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.startswith('checkpoint_') and f.endswith('.pth')]

# Extract epochs from file names and sort them
epochs = [int(re.search(r'checkpoint_(\d+).pth', f).group(1)) for f in checkpoint_files]
latest_epoch = max(epochs, default=0)  # Use default=0 to handle the case when the list is empty

# Load the latest checkpoint
if epochs:  # Check if the list is not empty
    latest_checkpoint_file = f'checkpoint_{latest_epoch}.pth'
    checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint_file)
    checkpoint = torch.load(checkpoint_path)

    # Restore the model and optimizer state
    model.load_state_dict(checkpoint['model_state_dict'])
    optimizer.load_state_dict(checkpoint['optimizer_state_dict'])

    # If you saved the epoch number and loss in the checkpoint, you can restore these too
    epoch = checkpoint['epoch']
    loss = checkpoint['loss']

    print(f"Checkpoint '{latest_checkpoint_file}' loaded. Resuming training from epoch {epoch}.")
else:
    print("No checkpoints found. Starting training from scratch.")
    epoch = 0  # Start from the first epoch
    loss = None  # Loss will be initialized during training

    # Initialize the weights of the model
    for m in model.modules():
        if isinstance(m, nn.Conv2d):
            # He initialization for layers with ReLU activation
            nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            if m.bias is not None:
                nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.BatchNorm2d):
            # Default initialization for batch normalization
            nn.init.constant_(m.weight, 1)
            nn.init.constant_(m.bias, 0)
        elif isinstance(m, nn.Linear):
            # Xavier initialization for fully connected layers
            nn.init.xavier_normal_(m.weight)
            nn.init.constant_(m.bias, 0)

# Load the dataset from the HDF5 file
hdf5_file = os.path.join(TRAINING_DATA_DIR, 'data.h5')

# Assuming you have already defined your complete dataset
dataset = MinecraftDataset(hdf5_file)

# # Define the lengths of the splits - 80% training, 10% validation, and 10% testing
# train_size = int(0.8 * len(dataset))
# val_test_size = len(dataset) - train_size
# val_size = int(0.5 * val_test_size)
# test_size = val_test_size - val_size

# # Randomly split the dataset into training, validation, and testing
# train_dataset, val_test_dataset = random_split(dataset, [train_size, val_test_size])
# val_dataset, test_dataset = random_split(val_test_dataset, [val_size, test_size])

train_dataset, val_dataset, test_dataset, _ = random_split(dataset, [100, 100, 100, len(dataset) - 300])

# Create DataLoaders for all datasets
BATCH_SIZE = 10
train_dataloader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE, shuffle=False)

# Print 1 batch of training data
for inputs, targets, name in train_dataloader:
    print(f"Inputs Shape: {inputs.shape}")
    print(f"Targets Shape: {targets.shape}")
    print(f"Name: {name}")
    break

# Log the model architecture to TensorBoard
example_dataloader = DataLoader(train_dataset, batch_size=1, shuffle=True)
example_inputs, _, _ = next(iter(example_dataloader))
example_inputs = example_inputs.to(device)
writer = SummaryWriter(f'runs/{experiment_name}')
writer.add_graph(model, example_inputs)

# Move the example inputs back to CPU to free up GPU memory
example_inputs = example_inputs.to('cpu')
del example_inputs  # Explicitly delete the example inputs
torch.cuda.empty_cache()  # Clear the cache if necessary

# Training loop
NUM_EPOCHS = 5000
for epoch in range(epoch, NUM_EPOCHS):
    model.train()
    train_loss = 0.0
    val_loss = 0.0

    train_bar = tqdm(train_dataloader, desc=f"Training Epoch {epoch + 1}/{NUM_EPOCHS}")
    for inputs, targets, prompt in train_bar:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        logits = model(inputs)

        # Bring the classes dimension to the end before flattening
        loss = F.cross_entropy(logits, targets)
        train_loss += loss.item()

        loss.backward()

        # Clip gradients
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)

        optimizer.step()

        # Update the progress bar with the latest loss
        train_bar.set_postfix(loss=loss.item())

        # Log the loss to TensorBoard
        writer.add_scalar('Loss/train', loss, epoch)

    # Calculate average training loss for the epoch
    train_loss /= len(train_dataloader)

    scheduler.step(loss)

    # Validation loop
    val_bar = tqdm(val_dataloader, desc=f"Validation Epoch {epoch + 1}/{NUM_EPOCHS}")
    model.eval()
    with torch.no_grad():
        for inputs, targets, _ in val_bar:
            inputs, targets = inputs.to(device), targets.to(device)

            logits = model(inputs)

            loss = F.cross_entropy(logits, targets)
            val_loss += loss.item()

            # Update the progress bar with the latest loss
            val_bar.set_postfix(loss=loss.item())

            # Log the loss to TensorBoard
            writer.add_scalar('Loss/val', loss, epoch)

    # Calculate average validation loss for the epoch
    val_loss /= len(val_dataloader)

    # Print training and validation loss
    print(f"Epoch {epoch+1}/{NUM_EPOCHS}, Training Loss: {train_loss:.4f}, Validation Loss: {val_loss:.4f}")

    # Save checkpoint
    if epoch % 5 == 0:
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'loss': loss
        }, os.path.join(checkpoint_dir, f"checkpoint_{epoch}.pth"))

writer.close()

MinecraftStructureGenerator(
  (fc1): Linear(in_features=1536, out_features=65536, bias=True)
  (bn1): BatchNorm1d(65536, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv1): Conv3d(128, 64, kernel_size=(3, 3, 3), stride=(1, 1, 1), padding=(1, 1, 1))
  (bn2): BatchNorm3d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv2): Conv3d(64, 32, kernel_size=(5, 5, 5), stride=(1, 1, 1), padding=(3, 3, 3))
  (bn3): BatchNorm3d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (conv3): Conv3d(32, 345, kernel_size=(7, 7, 7), stride=(1, 1, 1), padding=(5, 5, 5))
)
Number of parameters: 105124441
No checkpoints found. Starting training from scratch.
Inputs Shape: torch.Size([10, 1536])
Targets Shape: torch.Size([10, 64, 64, 64])
Name: ('72ccf061e5200bb34100ca424e78a005a0f2a318acdf9c8b6c608eac680686f4', 'f30f5603fe2f9ba393ddf4974f8ae2c0831402f0158b4502d452c953bc2ad3e5', 'a4fc69d7ef4898a6e123e5b0426537af30ae336441b0dc036505e94b04bba4f

Validation Epoch 1/5000: 100%|██████████| 10/10 [00:01<00:00,  8.00it/s, loss=2.56]


Epoch 1/5000, Training Loss: 2.9716, Validation Loss: 2.6701


Validation Epoch 2/5000: 100%|██████████| 10/10 [00:01<00:00,  9.98it/s, loss=2.81]


Epoch 2/5000, Training Loss: 1.2910, Validation Loss: 3.0719


...

Validation Epoch 116/5000: 100%|██████████| 10/10 [00:01<00:00,  9.07it/s, loss=4.28]


Epoch 116/5000, Training Loss: 0.1160, Validation Loss: 5.2181


Validation Epoch 117/5000: 100%|██████████| 10/10 [00:01<00:00,  9.74it/s, loss=4.34]


Epoch 117/5000, Training Loss: 0.1221, Validation Loss: 5.2437


Training Epoch 118/5000:  70%|███████   | 7/10 [00:02<00:00,  3.54it/s, loss=0.118] 

Inference

In [1]:
import os
import re

import torch
from openai import OpenAI

from common.file_paths import TRAINING_DATA_DIR
from model.model import MinecraftStructureGenerator
from converter.converter import RegionTensorConverter

# Assuming the constants are defined as in the training script
INPUT_EMBEDDING_SIZE = 1536
NUM_CLASSES = 345
OUTPUT_SIZE = [64, 64, 64]

# Initialize the model
model = MinecraftStructureGenerator(INPUT_EMBEDDING_SIZE, NUM_CLASSES, OUTPUT_SIZE)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Define the directory where checkpoints are saved
experiment_name = 'test14'
checkpoint_dir = f'checkpoints/{experiment_name}'

# List all checkpoint files
checkpoint_files = [f for f in os.listdir(checkpoint_dir) if f.startswith('checkpoint_') and f.endswith('.pth')]

# Extract epochs from file names and sort them
epochs = [int(re.search(r'checkpoint_(\d+).pth', f).group(1)) for f in checkpoint_files]
latest_epoch = max(epochs, default=0)  # Use default=0 to handle the case when the list is empty

# Load the trained model weights
latest_checkpoint_file = f'checkpoint_{latest_epoch}.pth'
print(f"Loading checkpoint '{latest_checkpoint_file}'...")
checkpoint_path = os.path.join(checkpoint_dir, latest_checkpoint_file)
checkpoint = torch.load(checkpoint_path, map_location=torch.device('cpu'))
model.load_state_dict(checkpoint['model_state_dict'])

# Set the model to evaluation mode
model.eval()

converter = RegionTensorConverter()

# Loop to take user input and perform inference
while True:
    user_input = input("Enter your text input (or type 'exit' to stop): ")
    if user_input.lower() == 'exit':
        break
    print(f"Input: {user_input}")

    # Get the embedding
    print("Getting embedding...")
    client = OpenAI()
    embedding = client.embeddings.create(input=user_input, model="text-embedding-ada-002").data[0].embedding
    input_tensor = torch.tensor(embedding).unsqueeze(0)  # Add batch dimension
    input_tensor = input_tensor.float()
    input_tensor = input_tensor.to(device)
    print(f"Embedding: {input_tensor.shape}")

    # Perform inference
    with torch.no_grad():
        print("Performing inference...")
        output = model(input_tensor)
        print(f"Output: {output.shape}")

    # Process result
    predicted_tokens = torch.argmax(output, dim=1)
    predicted_tokens = predicted_tokens.squeeze(0)
    print(f"Predicted Tokens: {predicted_tokens.shape}")

    # Convert the output tensor to a schematic
    print("Converting output tensor to schematic...")
    region = converter.tensor_to_region(predicted_tokens)
    print("Conversion complete.")

    # Save the schematic to a file
    print("Saving schematic to file...")
    # try:
    #     schematic = region.as_schematic()
    #     schematic.save('test.litematic')
    # except:
    #     print("Failed to save litematica schematic to file.")
    # try:
    #     structure_nbt = region.to_structure_nbt()
    #     structure_nbt.save('test.nbt')
    # except:
    #     print("Failed to save NBT schematic to file.")
    sponge_nbt = region.to_sponge_nbt()
    sponge_nbt.save(f'{user_input.lower().replace(" ", "")}.schem')
    print("Schematic saved to file.")

In [13]:
%load_ext autoreload
%autoreload 2

from pathlib import Path

from schempy.schematic import Block, Schematic, BlockEntity

schematic = Schematic.from_file(Path('sponge.3.schem'))
print(schematic.metadata)

# Usage example
# schematic = Schematic(width=10, height=10, length=10)
schematic.metadata['Description'] = "This is a schematic generated by SchemPy"

# Set a block at coordinates (x=1, y=2, z=3) to a specific value, e.g., 42
block = Block("minecraft:andesite")
schematic.set_block(1, 2, 3, block)
block = Block("minecraft:oak_planks")
schematic.set_block(0, 0, 0, block)

# Retrieve the block value at coordinates (x=1, y=2, z=3)
block = schematic.get_block(8, 9, 0)
print(block)
block_entity = BlockEntity("minecraft:chest", 0, 0, 0, {"LootTable": "minecraft:chests/simple_dungeon"})
schematic.add_block_entity(block_entity)

schematic.save_to_file(Path('example.schem'), 3)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
Compound({'Date': Long(1700278591692), 'WorldEdit': Compound({'Version': String('(unknown)'), 'EditingPlatform': String('enginehub:fabric'), 'Origin': IntArray([Int(0), Int(0), Int(0)]), 'Platforms': Compound({'enginehub:fabric': Compound({'Name': String('Fabric-Official'), 'Version': String('7.3.0-beta-02+e11f161')})})})})
minecraft:air
